In [1]:
%load_ext autoreload
%autoreload 2

import torch
import gpytorch
import pinot
import math

Using backend: pytorch


In [2]:
data = pinot.data.esol()
ds_tr, ds_te = pinot.data.utils.split(data, [4, 1])

batch_size = 32
ds_tr_onebatch = pinot.data.utils.batch(ds_tr, len(ds_tr))
ds_tr_batched  = pinot.data.utils.batch(ds_tr, 32)
ds_te_onebatch = pinot.data.utils.batch(ds_te, len(ds_te))

/home/duc/anaconda3/envs/torch-research/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


# Define the model

In [3]:
def rmse(y, yhat):
    return torch.sqrt(torch.mean((y.flatten()-yhat.flatten())**2))

In [4]:
class GaussianProcessLayer(gpytorch.models.ApproximateGP):
    def __init__(self, in_features, grid_bounds=(-10., 10.), grid_size=150):
        
        inducing_points = torch.rand(grid_size, in_features)

        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            grid_size
        )

        # LMCVariationalStrategy for introducing correlation among tasks if we want MultiTaskGP
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        )
    
        super().__init__(variational_strategy)
        
        self.covar_module = gpytorch.kernels.RBFKernel()
#         self.covar_module = gpytorch.kernels.ScaleKernel(
#             gpytorch.kernels.RBFKernel(
#                 lengthscale_prior=gpytorch.priors.SmoothedBoxPrior(
#                     math.exp(-1), math.exp(1), sigma=0.1, transform=torch.exp
#                 )
#             )
#         )
        self.mean_module = gpytorch.means.LinearMean(in_features)
        self.grid_bounds = grid_bounds

    def forward(self, x):
#         print("GP.forward.x.shape", x.shape)
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)
    
        
class DKLModel(gpytorch.Module):
    def __init__(self, feature_extractor, num_dim, grid_bounds=(-10., 10.)):
        super(DKLModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.gp_layer = GaussianProcessLayer(in_features=num_dim, grid_bounds=grid_bounds)
        self.grid_bounds = grid_bounds
        self.num_dim = num_dim

    def forward(self, x):
        features = self.feature_extractor(x)
#         features = gpytorch.utils.grid.scale_to_bounds(features, self.grid_bounds[0], self.grid_bounds[1])
#         print(features.shape)
        # This next line makes it so that we learn a GP for each feature
        res = self.gp_layer(features)
        return res


In [5]:
gpu = torch.device("cuda")
cpu = torch.device("cpu")
device = cpu


feature_extractor = pinot.representation.Sequential(
        pinot.representation.dgl_legacy.gn(kwargs={"allow_zero_in_degree":True}),
        [32, 'tanh', 32, 'tanh', 32, 'tanh']).to(device)

model = DKLModel(feature_extractor, num_dim=32).to(device)
likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
mll = gpytorch.mlls.VariationalELBO(likelihood, model.gp_layer, num_data=902).to(device)


lr = 1e-3
optimizer = torch.optim.Adam([
    {'params': model.feature_extractor.parameters(), 'weight_decay': 1e-4},
    {'params': model.gp_layer.parameters(), 'lr': lr*0.1},
    {'params': likelihood.parameters()},
], lr=lr)


for n in range(5000):
    total_loss = 0.
    for (g, y) in ds_tr_onebatch:
        optimizer.zero_grad()
        output = model(g.to(device))
        loss = -mll(output, y.flatten().to(device))
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
    
    if n % 100 == 0:
        train_rmse = rmse(model(ds_tr_onebatch[0][0]).mean.cpu(), ds_tr_onebatch[0][1])
        test_rmse = rmse(model(ds_te_onebatch[0][0]).mean.cpu(), ds_te_onebatch[0][1])
        print (total_loss, train_rmse, test_rmse)



58.350440979003906 tensor(32.9760, grad_fn=<SqrtBackward>) tensor(32.3698, grad_fn=<SqrtBackward>)
3.7204818725585938 tensor(1.7835, grad_fn=<SqrtBackward>) tensor(1.7496, grad_fn=<SqrtBackward>)
3.073115825653076 tensor(1.5075, grad_fn=<SqrtBackward>) tensor(1.4545, grad_fn=<SqrtBackward>)
2.5964291095733643 tensor(1.2679, grad_fn=<SqrtBackward>) tensor(1.1695, grad_fn=<SqrtBackward>)
2.305521249771118 tensor(1.0967, grad_fn=<SqrtBackward>) tensor(0.9583, grad_fn=<SqrtBackward>)
2.2024986743927 tensor(1.0302, grad_fn=<SqrtBackward>) tensor(0.8661, grad_fn=<SqrtBackward>)
2.17500901222229 tensor(1.0126, grad_fn=<SqrtBackward>) tensor(0.8377, grad_fn=<SqrtBackward>)
2.1603469848632812 tensor(1.0036, grad_fn=<SqrtBackward>) tensor(0.8258, grad_fn=<SqrtBackward>)
2.147128105163574 tensor(0.9957, grad_fn=<SqrtBackward>) tensor(0.8181, grad_fn=<SqrtBackward>)
2.1337296962738037 tensor(0.9875, grad_fn=<SqrtBackward>) tensor(0.8117, grad_fn=<SqrtBackward>)
2.1198956966400146 tensor(0.9791, gr

KeyboardInterrupt: 

In [12]:
# print(model(ds_tr_onebatch[0][0]).mean)


print(train_rmse, test_rmse)

tensor(1.0345, grad_fn=<MeanBackward0>) tensor(0.7299, grad_fn=<MeanBackward0>)
